# Factor analysis using Bi-Cross validation

## Overview

FIXME: cite Jingshu Wang's paper (2016?)

## Input and output

Same interface as [PEER factor analysis](PEER_factor.html). 

## Minimal working example

**FIXME: please add it and upload to Google Drive a folder APEX_factor containing the MWE data**

In [ ]:
[global]
# The output directory for generated files. MUST BE FULL PATH
parameter: cwd = path
# The molecular phenotype matrix
parameter: phenoFile = path
# The covariate file
parameter: covFile = path
# The VCF file, required by APEX
# https://github.com/hsun3163/neuro-apex/issues/1
# https://github.com/cumc/xqtl-pipeline/issues/138
parameter: vcfFile = path
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 8
# Software container option
parameter: container = ""
parameter: name = ""
# N PEER factors, If specified as 0 it will use APEX default
parameter: N = 0

# Default values from PEER:
## The number of max iteration

parameter: iteration = 10

In [3]:
[BiCV]
input: phenoFile, covFile, vcfFile
output: f'{cwd:a}/{name}.BiCV.cov.gz'
task: trunk_workers = 1, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
bash: container=container, expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    apex factor \
        --out $[_output[0]:nn] \
        --iter $[iteration] \
        $[("--factors %s" % N) if N > 0 else ""] \
        --bed $[_input[0]] \
        --vcf $[_input[2]] \
        --cov $[_input[1]]

    gzip -f $[_output[0]]